In [43]:
from tools import gemini2_flash,gpt4o_mini
import pandas as pd
import json
from ast import literal_eval
import time





import vertexai
from vertexai import generative_models
from vertexai.generative_models import GenerativeModel, Part
import os

def initialize_vertexai(
    credential_json_path: str,
) -> None:
    """
    Initialize the Vertex AI client.
    """
    with open(credential_json_path, "r", encoding="utf-8") as f:
        credentials = json.load(f)

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_json_path
    vertexai.init(project=credentials["project_id"], location="us-central1")


GEMINI_SAFETY_SETTING = [
    generative_models.SafetySetting(
        category=generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    generative_models.SafetySetting(
        category=generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    generative_models.SafetySetting(
        category=generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
]

initialize_vertexai("./preceptorai-shared-research.json")
client = GenerativeModel("gemini-2.0-flash") #gemini-2.5-flash-preview-04-17

def gemini_j(text:str):
    responses = client.generate_content(
            contents=[text],
            generation_config={
                "temperature": 0.1,
            },
            safety_settings=GEMINI_SAFETY_SETTING,
        )


    text_result = responses.candidates[0].content.text
    return text_result


In [34]:
df.head()

,UID,Name,Link (only from pdb),CID,Source,Description (only from pdb),Event Childhood,Event Teenage,Event Pre-Hero,Event Hero (1),Event Hero (2),Event Hero (3),Dilemma(1) Save vs. Sacrifice (Personal Sacrifice vs. Greater Good),Dilemma (2) Hero or Villain (Morality and Identity Crisis),Dilemma(3) Duty vs. Personal Desire,Dilemma(4) The Ends Justify the Means vs. Moral Code,True event,Human label,annotation
0,1,Tony Stark “Iron Man”,https://www.personality-database.com/profile/1...,148,"['MCU: The Heroes', 'Wreck-It Ralph (Franchise)']",Tony Stark was a character portrayed by #Rober...,After graduating from MIT at age 17 with summa...,"when you was 21, your parents are killed in a ...",when after break free from imprisoned by a ter...,Now that you was publicly revealed yourself as...,"During the Battle of New York, the Chitauri in...","In the ultimate battle against Thanos, the Ave...",Title: A Rift in Reality\n\nScene:\nThe sky ab...,"Scene: The Stark Industries Lab\n\nTony Stark,...",Title: The Stark Dichotomy\n\n---\n\nScene: St...,Scene: The Stark Tower Laboratory\n\nTony Star...,B A A B C A,NaN,NaN
1,1,Tony Stark “Iron Man”,https://www.personality-database.com/profile/1...,1982,"['Marvel Comics', 'Marvel vs. Capcom']","Iron Man (Anthony Edward ""Tony"" Stark) is a fi...",which pivotal decision did you that significan...,what pivotal decision did you make that influe...,Before fully committing to becoming a superher...,"During the events of the ""Civil War,"" you made...","when becomes the head of S.H.I.E.L.D., you fa...",During a mission to stop an alien invasion led...,Title: Iron Man: The Reactor's Reckoning\n\nSi...,Title: The Moral Quandary of Tony Stark: Hero ...,Scene:\n\nThe sun is setting over the New York...,Title: The Dilemma of Iron Man: The Price of P...,B C C A C D,NaN,NaN
2,1,Tony Stark,https://www.personality-database.com/profile/3...,376301,['What If...? (2021)'],"Anthony Edward ""Tony"" Stark was a billionaire ...",As a child prodigy who built your first circui...,"when you was 21, your parents are killed in a ...",After a near-death experience in Kunar where E...,"During a party at your mansion, Stevens shares...","When working on Project Liberator, Stevens rev...",You've just learned that Stevens killed your f...,"Title: ""A Hero’s Choice""\n\nScene:\n\nThe bust...","Title: ""Tony Stark: The Line Between Hero and ...",Title: The Stark Dilemma: A New Path\n\nScene:...,Title: The Stark Choice\n\nScene: An Isolated ...,B A A B A C,NaN,NaN
3,2,Spider-Man,https://www.personality-database.com/profile/2...,2589,"['MCU: The Heroes', 'Spider-Man : No Way Home']",After getting bitten by a scientifically-alter...,During your childhood visit to the Stark Expo ...,how do you balances your double life as Spide...,"Before you gained your spider powers, you had ...","During a mid-air battle, Spider-Man was caught...","While at the hotel for the Academic Decathlon,...",You've reached the top of the Washington Monum...,"Title: The Choice\n\nScene: \nNew York City, ...",Title: The Shattered Mirror\n\nScene: A Roofto...,Scene:\n\nThe city of New York bustles as usua...,Title: The Shattered Choice\n\nScene:\n\nThe n...,C C A B B A,NaN,NaN
4,2,Spider-Man,https://www.personality-database.com/profile/9...,920,"['The Amazing Spider-Man Duology', 'MCU: The H...",Peter Benjamin Parker is the superhero vigilan...,"When your parents, Richard and Mary Parker is ...","As Peter Parker, before gaining your spider po...",You've recently been gifted with amazing spide...,"you've discovered that your mentor, Dr. Curt C...",you find yourself in a difficult situation wh...,\nAfter the tragic event of being unable to sa...,Scene:\n\nIn the bustling streets of New York ...,Scene:\n\nThe city of New York sprawls beneath...,Situation: Duty vs. Personal Desire\n\nScene: ...,"Title: The Dark Web\n\nScene: New York City, l...",A B B D B C,NaN,NaN


In [ ]:

for file in ["canon_claude_rep1.jsonl","canon_gemini2_rep1.jsonl","canon_gemini2_rep1_fixed.jsonl","canon_o3_rep1_fixed.jsonl","canon_r1_rep1_fixed_1.jsonl","canon_v3_rep1.jsonl"]:
    with open(f"/Users/kunkerdthaisong/charactor_ai/generated_results/{file}", "r") as f:
        data = f.readlines()

    df=pd.read_csv("/Users/kunkerdthaisong/charactor_ai/all_character_data-canon.csv")
    #data_df=pd.read_json("/Users/kunkerdthaisong/charactor_ai/generated_results/claude/canon_claude_rep1.jsonl",lines=True)

    predicted_l=[]
    gt_l=[]
    c=0
    for i,row in df.iterrows():#data_df.iterrows():
        cid=row["CID"]
        true_answers=row["True event"]
        
        print(cid)
        # print(c)
        for answer in true_answers.split(" "):

            #print(answer)
            gt_l.append(answer)
            time.sleep(0.2)
            
            if answer not in ["B","A","C","D"]:
                predicted_l.append("_")
            else:
                d=literal_eval(data[c])

                if answer not in ["A","B","C","D"]:
                    predicted_l.append("_")
                    c+=1
                else:
                    predicted=d["answers"]
                    c+=1
                    answer_letter=""
                    answer_letter=gemini_j(text=f"you are must capture only one true answer return A or B or C or D \n here the input:{predicted}").strip()
                    if answer_letter not in ["A","B","C","D"]:
                        predicted_l.append("O")
                    else:
                        predicted_l.append(answer_letter)


    # data_df["predicted"]=predicted_l

    from sklearn.metrics import accuracy_score

    # Remove '_' from labels


    # Split by index modulo 6
    gt_l_child     = [gt_l[i] for i in range(len(gt_l)) if i % 6 == 0]
    pred_l_child   = [predicted_l[i] for i in range(len(predicted_l)) if i % 6 == 0]
    gt_l_child     = [label for label in gt_l_child if label != '_']
    pred_l_child   = [label for label in pred_l_child if label != '_']


    gt_l_teen      = [gt_l[i] for i in range(len(gt_l)) if i % 6 == 1]
    pred_l_teen    = [predicted_l[i] for i in range(len(predicted_l)) if i % 6 == 1]
    gt_l_teen     = [label for label in gt_l_teen if label != '_']
    pred_l_teen   = [label for label in pred_l_teen if label != '_']

    gt_l_pre       = [gt_l[i] for i in range(len(gt_l)) if i % 6 == 2]
    pred_l_pre     = [predicted_l[i] for i in range(len(predicted_l)) if i % 6 == 2]
    gt_l_pre     = [label for label in gt_l_pre if label != '_']
    pred_l_pre   = [label for label in pred_l_pre if label != '_']

    gt_l_hero123   = [gt_l[i] for i in range(len(gt_l)) if i % 6 in [3, 4, 5]]
    pred_l_hero123 = [predicted_l[i] for i in range(len(predicted_l)) if i % 6 in [3, 4, 5]]
    gt_l_hero123   = [label for label in gt_l_hero123 if label != '_']
    pred_l_hero123 = [label for label in pred_l_hero123 if label != '_']


    gt_l_ = [label for label in gt_l if label != '_']
    predicted_l_ = [label for label in predicted_l if label != '_']




    # Compute accuracy
    acc_all     = accuracy_score(gt_l_, predicted_l_)
    acc_child   = accuracy_score(gt_l_child, pred_l_child)
    acc_teen    = accuracy_score(gt_l_teen, pred_l_teen)
    acc_pre     = accuracy_score(gt_l_pre, pred_l_pre)
    acc_hero123 = accuracy_score(gt_l_hero123, pred_l_hero123)

    # Save results
    df_result = pd.DataFrame([{
        "model": f"{os.path.basename(file)}",
        "acc_child": acc_child,
        "acc_teen": acc_teen,
        "acc_pre": acc_pre,
        "acc_hero123": acc_hero123,
        "acc_all": acc_all,
    }])

    df_result.to_csv(f"{os.path.basename(file)}.csv", index=False)

148
1982
376301
2589
920
15917
155
1983
154
2004
344985
1800221
20233
2003
1986
2337
1748
2440
153
417034
2441
1754
1987
9046
844
403126
9049
1993
375742
476
2267
159239
800
2013
2005
383180
4345
2011
388252
133165
398132
2006
344993
2588
2010
398398
1994
2648
1992
83252
14668
1996
1752
1995
506747
83268


In [78]:
len(gt_l),len(predicted_l),len(data_df)

(336, 336, 288)

In [62]:
len(predicted_l),len(data_df),len(gt_l)

(249, 288, 282)

In [16]:
data_df.to_csv("canon_claude_rep1.csv",index=False)

In [15]:
from sklearn.metrics import accuracy_score

data_df['predicted'] = data_df['predicted'].str.strip()
accuracy = accuracy_score(data_df['true_label'], data_df['predicted'])
print(accuracy)

0.6319444444444444


In [ ]:
data_df[""]

0         148
1         148
2         148
3         148
4         148
        ...  
283    506747
284    506747
285     83268
286     83268
287     83268
Name: CID, Length: 288, dtype: int64

In [24]:
288/6

48.0